In [1]:
# Import necessary modules
from loguru import logger
from datetime import datetime
import pandas as pd
from src.elt.utils import connect_mongodb
from bson.objectid import ObjectId

2025-09-01 00:33:13.145 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /Users/Edward/DS/Projects/book-club


In [2]:
def count_books_read_this_year():
    """
    Connects to the MongoDB database and counts the number of books read
    by all users in the current year.
    """
    db, client = connect_mongodb()

    # Get the current year
    current_year = datetime.now().year

    # Define the aggregation pipeline
    pipeline = [
        # Match documents with a completed date in the current year
        {
            "$match": {
                "date_completed": {
                    "$gte": datetime(current_year, 1, 1),
                    "$lt": datetime(current_year + 1, 1, 1)
                }
            }
        },
        # Group all matched documents and count them
        {
            "$group": {
                "_id": None,
                "total_books_read": {"$count": {}}
            }
        }
    ]

    try:
        # Execute the pipeline
        results = list(db["user_reads"].aggregate(pipeline))
        
        # Extract the total count
        if results:
            total_books_read = results[0]["total_books_read"]
            logger.success(f"A total of {total_books_read} books were read this year.")
        else:
            logger.info("No books were read this year.")
            
    except Exception as e:
        logger.error(f"An error occurred during aggregation: {e}")
    finally:
        client.close()
        logger.info("MongoDB connection closed.")

if __name__ == "__main__":
    count_books_read_this_year()



2025-08-31 11:09:27.080 | INFO     | src.elt.utils:connect_mongodb:24 - Successfully connected to MongoDB
2025-08-31 11:09:27.139 | SUCCESS  | __main__:count_books_read_this_year:38 - A total of 44 books were read this year.
2025-08-31 11:09:27.166 | INFO     | __main__:count_books_read_this_year:46 - MongoDB connection closed.


In [8]:
def books_read_by_month():
    """
    Connects to the MongoDB database, counts the number of books read
    by all users each month, and displays the result in a Pandas DataFrame.
    """
    db, client = connect_mongodb()

    # Define the aggregation pipeline to count books read per month
    pipeline = [
        # Match documents that have a completed date and are marked as completed
        {
            "$match": {
                "date_completed": {"$ne": None},
                "current_rstatus": "Read"
            }
        },
        # Group documents by year and month of the completion date
        {
            "$group": {
                "_id": {
                    "year": {"$year": "$date_completed"},
                    "month": {"$month": "$date_completed"}
                },
                "books_read": {"$count": {}}
            }
        },
        # Sort the results chronologically by year and then by month
        {
            "$sort": {
                "_id.year": 1,
                "_id.month": 1
            }
        }
    ]

    try:
        # Execute the aggregation pipeline
        results = list(db["user_reads"].aggregate(pipeline))
        
        if not results:
            logger.info("No completed books found. The DataFrame will be empty.")
            df = pd.DataFrame(columns=['Year', 'Month', 'Books Read'])
        else:
            # Create a list of dictionaries in a format suitable for a DataFrame
            df_data = [
                {
                    "Year": r["_id"]["year"],
                    "Month": r["_id"]["month"],
                    "Books Read": r["books_read"]
                } for r in results
            ]
            
            # Create the Pandas DataFrame
            df = pd.DataFrame(df_data)
            
        logger.success("Successfully created the DataFrame:")
        return df
        
    except Exception as e:
        logger.error(f"An error occurred during aggregation or DataFrame creation: {e}")
    finally:
        client.close()
        logger.info("MongoDB connection closed.")

df = books_read_by_month()
df

2025-09-01 01:08:26.355 | INFO     | src.elt.utils:connect_mongodb:24 - Successfully connected to MongoDB
2025-09-01 01:08:26.388 | SUCCESS  | __main__:books_read_by_month:56 - Successfully created the DataFrame:
2025-09-01 01:08:26.416 | INFO     | __main__:books_read_by_month:63 - MongoDB connection closed.


,Year,Month,Books Read
0,2023,5,1
1,2024,3,2
2,2024,5,2
3,2024,7,1
4,2024,8,1
5,2024,9,2
6,2024,11,4
7,2024,12,6
8,2025,1,8
9,2025,2,7


In [ ]:
# Define the creator's custom ID to search for
creator_object_id_to_find = "68b21e02100b8ea7b076efd9"

def get_books_by_creator(creator_object_id):
    """
    Connects to the MongoDB database and returns a Pandas DataFrame of
    all books associated with a given creator ObjectId.
    """
    try:
        # Connect to MongoDB
        db, client = connect_mongodb()

        # Step 1: Query the 'books' collection using the creator's ObjectId
        # This query leverages the new data model where both the ID and name are embedded.
        books_collection = db["books"]
        query_results = books_collection.find({"author._id": ObjectId(creator_object_id)})
        
        # Step 2: Convert the query results into a Pandas DataFrame
        books_df = pd.DataFrame(list(query_results))
        
        # Clean up the author column for a more readable output
        if not books_df.empty:
            books_df['author_name'] = books_df['author'].apply(
                lambda authors: ', '.join([a['author_name'] for a in authors if 'author_name' in a])
            )
            # Drop the original 'author' column and '_id' for clarity
            books_df = books_df.drop(columns=['author', '_id'])
        
        logger.info(f"Found {len(books_df)} books written by this author.")
        return books_df

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return pd.DataFrame()
    finally:
        if 'client' in locals():
            client.close()
            logger.info("MongoDB connection closed.")

books_df = get_books_by_creator(creator_object_id_to_find)
books_df

2025-08-31 11:11:45.602 | INFO     | src.elt.utils:connect_mongodb:24 - Successfully connected to MongoDB
2025-08-31 11:11:45.633 | INFO     | __main__:get_books_by_creator:29 - Found 8 books written by this creator.
2025-08-31 11:11:45.666 | INFO     | __main__:get_books_by_creator:38 - MongoDB connection closed.


,title,genre_name,collection,collection_index,description,first_publication_date,tags,formats,awards,author_name
0,Baptism of Fire,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,3.0,The Wizards Guild has been shattered by a coup...,1996-01-01,"[68b21e19100b8ea7b076f25a, 68b21e19100b8ea7b07...","[{'format_name': 'Kindle', 'isbn_13': '978-057...",NaN,Andrzej Sapkowski
1,Blood of Elves,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,1.0,"For more than a hundred years, humans, dwarves...",1994-01-01,[68b21e19100b8ea7b076f25a],"[{'format_name': 'Kindle', 'isbn_13': '978-057...","[{'award_id': 68b21e0c100b8ea7b076f10d, 'categ...",Andrzej Sapkowski
2,Lady of the Lake,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,5.0,After walking through a portal in the Tower of...,1999-01-01,"[68b21e19100b8ea7b076f25a, 68b21e19100b8ea7b07...","[{'format_name': 'Kindle', 'asin': 'B01M4GH0GS...",NaN,Andrzej Sapkowski
3,Season of Storms,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,0.2,Geralt. The witcher whose mission is to protec...,2013-11-06,NaN,"[{'format_name': 'Kindle', 'asin': 'B076PFJ7K7...","[{'award_id': 68b21e0c100b8ea7b076f10d, 'categ...",Andrzej Sapkowski
4,Sword of Destiny: Tales of the Witcher,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,0.7,"Geralt of Rivia is a Witcher, a man whose magi...",1992-05-21,[68b21e19100b8ea7b076f25a],"[{'format_name': 'Kindle', 'asin': 'B00U68KUE2...",NaN,Andrzej Sapkowski
5,The Last Wish,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,0.5,"Geralt of Rivia is a Witcher, a man whose magi...",1993-01-31,"[68b21e19100b8ea7b076f25a, 68b21e19100b8ea7b07...","[{'format_name': 'Kindle', 'asin': 'B0043M6712...","[{'award_id': 68b21e0c100b8ea7b076f12f, 'categ...",Andrzej Sapkowski
6,Time of Contempt,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,2.0,The kings and armies are manoeuvring for posit...,1995-01-01,"[68b21e19100b8ea7b076f25a, 68b21e19100b8ea7b07...","[{'format_name': 'Kindle', 'edition': '1', 'as...",NaN,Andrzej Sapkowski
7,Tower of the Swallow,"[Fiction, Fantasy, Adventure, High Fantasy]",68b21e09100b8ea7b076f0d1,4.0,"The world has fallen into war. Ciri, the child...",1997-01-01,"[68b21e19100b8ea7b076f25a, 68b21e19100b8ea7b07...","[{'format_name': 'Kindle', 'asin': 'B019CSNPV6...","[{'award_id': 68b21e0c100b8ea7b076f12a, 'year'...",Andrzej Sapkowski


In [14]:
creator_object_id_to_find = "68b21e02100b8ea7b076efd5"

def get_books_by_creator(creator_object_id):
    """
    Connects to the MongoDB database and returns a Pandas DataFrame of
    all books associated with a given creator ObjectId.
    """
    try:
        # Connect to MongoDB
        db, client = connect_mongodb()

        # Step 1: Query the 'books' collection using the creator's ObjectId
        # This query leverages the new data model where both the ID and name are embedded.
        books_collection = db["books"]
        query_results = books_collection.find(
            {"author._id": ObjectId(creator_object_id)},
            {"title": 1, "genre_name": 1, "first_publication_date": 1, "author": 1}
        )
        
        # Step 2: Convert the query results into a Pandas DataFrame
        books_df = pd.DataFrame(list(query_results))
        
        # Clean up the author column for a more readable output
        if not books_df.empty:
            books_df['author_name'] = books_df['author'].apply(
                lambda authors: ', '.join([a['author_name'] for a in authors if 'author_name' in a])
            )
            # Drop the original 'author' column and '_id' for clarity
            books_df = books_df.drop(columns=['author', '_id'])
        
        logger.info(f"Found {len(books_df)} books written by this creator.")
        return books_df

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return pd.DataFrame()
    finally:
        if 'client' in locals():
            client.close()
            logger.info("MongoDB connection closed.")

books_df = get_books_by_creator(creator_object_id_to_find)
books_df

2025-08-31 11:14:15.591 | INFO     | src.elt.utils:connect_mongodb:24 - Successfully connected to MongoDB
2025-08-31 11:14:15.624 | INFO     | __main__:get_books_by_creator:31 - Found 2 books written by this creator.
2025-08-31 11:14:15.656 | INFO     | __main__:get_books_by_creator:40 - MongoDB connection closed.


,title,genre_name,author_name
0,Alien Clay,"[Fiction, Sci-Fi, Fantasy, Space, Aliens]",Adrian Tchaikovsky
1,Children of Time,"[Fiction, Sci-Fi, Space]",Adrian Tchaikovsky
